In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

from xgboost import XGBRegressor

from scipy.stats import randint as sp_randint

In [2]:
train = pd.read_csv('train.csv')
train

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,8405,107.39,ak,s,as,c,d,aa,d,q,...,1,0,0,0,0,0,0,0,0,0
4205,8406,108.77,j,o,t,d,d,aa,h,h,...,0,1,0,0,0,0,0,0,0,0
4206,8412,109.22,ak,v,r,a,d,aa,g,e,...,0,0,1,0,0,0,0,0,0,0
4207,8415,87.48,al,r,e,f,d,aa,l,u,...,0,0,0,0,0,0,0,0,0,0


This dataset contains an anonymized set of variables, each representing a custom feature in a Mercedes car. For example, a variable could be 4WD, added air suspension, or a head-up display.

The ground truth is labeled ‘y’ and represents the time (in seconds) that the car took to pass testing for each variable.

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4209 entries, 0 to 4208
Columns: 378 entries, ID to X385
dtypes: float64(1), int64(369), object(8)
memory usage: 12.1+ MB


In [4]:
train.describe()

,ID,y,X10,X11,X12,X13,X14,X15,X16,X17,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
count,4209.000000,4209.000000,4209.000000,4209.0,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,...,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000
mean,4205.960798,100.669318,0.013305,0.0,0.075077,0.057971,0.428130,0.000475,0.002613,0.007603,...,0.318841,0.057258,0.314802,0.020670,0.009503,0.008078,0.007603,0.001663,0.000475,0.001426
std,2437.608688,12.679381,0.114590,0.0,0.263547,0.233716,0.494867,0.021796,0.051061,0.086872,...,0.466082,0.232363,0.464492,0.142294,0.097033,0.089524,0.086872,0.040752,0.021796,0.037734
min,0.000000,72.110000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2095.000000,90.820000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4220.000000,99.150000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,6314.000000,109.010000,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,8417.000000,265.320000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
X, y = train.drop(['ID', 'y'], axis=1), train['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
X.corr().ge(0.7)

,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
X10,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
X11,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
X12,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
X13,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
X14,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X380,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
X382,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
X383,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
X384,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [7]:
cols_num = X.select_dtypes(include = ['int64', 'float64']).columns
cols_cat = X.select_dtypes(include = ['object']).columns

trans_num = StandardScaler()

In [8]:

preprocessor = ColumnTransformer(transformers=[('numerical', StandardScaler(), cols_num),
                                               ('categorical', OneHotEncoder(drop ='first',
                                                                             handle_unknown='ignore'), cols_cat)])

In [9]:
pipe = Pipeline(steps = [('preprocessor', preprocessor), ('model', None)])

In [10]:
param_grid = [
    {
        'preprocessor__categorical' : [None]
        'model__n_estimators' : sp_randint(200, 500),
        'model__max_depth' : np.arange(1, 11, 1),
        'model__min_samples_split' : np.arange(2, 10, 1),
        'model__min_samples_leaf' : sp_randint(1, 12),
        'model__ccp_alpha' : [0, 0.0001, 0.001, 0.01, 0.1, 5, 10, 50, 100, 500, 1000],
        'model': [RandomForestRegressor()]
    },
    {
        'preprocessor__numerical' : [StandardScaler(), MinMaxScaler()],
        'model__learning_rate' : ['constant', 'optimal', 'invscaling', 'adaptive'],
        'model__alpha' : [0, 0.0001, 0.001, 0.01, 0.1, 5, 10, 50, 100, 500, 1000],
        'model': [SGDRegressor()]
    },
    {
        'preprocessor__categorical' : [None]
        'model__n_estimators' : sp_randint(10, 100),
        'model__max_depth': np.arange(2, 9),
        'model__learning_rate': np.arange(0.01, 0.3, 0.01),
        'model__colsample_bytree': np.arange(0.1, 1, 0.1),
        'model__gamma': np.arange(0, 20, 2),
        'model': [XGBRegressor()]
    }

]
grid = RandomizedSearchCV(pipe, param_grid, n_iter=1000, scoring='r2')
grid.fit(X_train, y_train)

D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [0, 1, 2, 5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [0, 2, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [0, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [0, 2, 5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarn

D:\anaconda\envs\merc\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1527: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [0, 2, 5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1527: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [2, 5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: U

D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [0, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [0, 2, 5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [2, 5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [0, 1, 2, 5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning

D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [2, 5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [0, 1, 2, 5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [0, 2, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [0, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning

D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [0, 2, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [0, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [0, 2, 5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [2, 5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: F

...

D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [0, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [0, 2, 5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [2, 5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [0, 1, 2, 5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning

D:\anaconda\envs\merc\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1527: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [0, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1527: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [0, 2, 5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\linear_model\_stochastic_gradient

D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [0, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [0, 2, 5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [2, 5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1527: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarni

D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [2, 5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
D:\anaconda\envs\merc\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
20 fits failed out of a total of 5000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "D:\anaconda\envs\merc\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\anaconda\envs\merc\lib\site-packages\sklearn\pipeline.py", line 382, in fit
  

RandomizedSearchCV(estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('numerical',
                                                                               StandardScaler(),
                                                                               Index(['X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19',
       ...
       'X375', 'X376', 'X377', 'X378', 'X379', 'X380', 'X382', 'X383', 'X384',
       'X385'],
      dtype='object', length=368)),
                                                                              ('categorical',
                                                                               OneHotEncoder(drop='first',
                                                                                             handle_unknown='...
                                         'model__gamma': array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18]),
                                         'model__learning_rate': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29]),
                                         'model__max_depth': array([2, 3, 4, 5, 6, 7, 8]),
                                         'model__n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000014252D2ACB0>}],
                   scoring='r2')

In [11]:
pd.set_option('display.max_colwidth', 300)
pd.DataFrame(grid.cv_results_).sort_values(by = 'rank_test_score')[['params',
                                                                    'mean_fit_time',
                                                                    'mean_test_score']]

,params,mean_fit_time,mean_test_score
417,"{'model': RandomForestRegressor(ccp_alpha=0, max_depth=4, min_samples_leaf=2, min_samples_split=9, n_estimators=343), 'model__ccp_alpha': 0, 'model__max_depth': 4, 'model__min_samples_leaf': 2, 'model__min_samples_split': 9, 'model__n_estimators': 343}",6.298553,5.648953e-01
917,"{'model': RandomForestRegressor(ccp_alpha=0, max_depth=4, min_samples_leaf=2, min_samples_split=9, n_estimators=343), 'model__ccp_alpha': 0.0001, 'model__max_depth': 4, 'model__min_samples_leaf': 2, 'model__min_samples_split': 6, 'model__n_estimators': 239}",4.364741,5.638466e-01
2,"{'model': RandomForestRegressor(ccp_alpha=0, max_depth=4, min_samples_leaf=2, min_samples_split=9, n_estimators=343), 'model__ccp_alpha': 0.001, 'model__max_depth': 5, 'model__min_samples_leaf': 2, 'model__min_samples_split': 3, 'model__n_estimators': 375}",8.765150,5.637844e-01
640,"{'model': RandomForestRegressor(ccp_alpha=0, max_depth=4, min_samples_leaf=2, min_samples_split=9, n_estimators=343), 'model__ccp_alpha': 0.001, 'model__max_depth': 4, 'model__min_samples_leaf': 2, 'model__min_samples_split': 5, 'model__n_estimators': 210}",3.750383,5.637239e-01
384,"{'model': RandomForestRegressor(ccp_alpha=0, max_depth=4, min_samples_leaf=2, min_samples_split=9, n_estimators=343), 'model__ccp_alpha': 0.01, 'model__max_depth': 4, 'model__min_samples_leaf': 3, 'model__min_samples_split': 6, 'model__n_estimators': 483}",8.563726,5.633008e-01
...,...,...,...
679,"{'model': SGDRegressor(), 'model__alpha': 0.0001, 'model__learning_rate': 'optimal', 'preprocessor__numerical': StandardScaler()}",0.328525,-2.069015e+24
493,"{'model': SGDRegressor(), 'model__alpha': 0, 'model__learning_rate': 'optimal', 'preprocessor__numerical': MinMaxScaler()}",0.037706,NaN
776,"{'model': SGDRegressor(), 'model__alpha': 0, 'model__learning_rate': 'optimal', 'preprocessor__numerical': MinMaxScaler()}",0.037306,NaN
814,"{'model': SGDRegressor(), 'model__alpha': 0, 'model__learning_rate': 'optimal', 'preprocessor__numerical': MinMaxScaler()}",0.036304,NaN


In [12]:
grid.best_estimator_

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical', StandardScaler(),
                                                  Index(['X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19',
       ...
       'X375', 'X376', 'X377', 'X378', 'X379', 'X380', 'X382', 'X383', 'X384',
       'X385'],
      dtype='object', length=368)),
                                                 ('categorical',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  Index(['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8'], dtype='object'))])),
                ('model',
                 RandomForestRegressor(ccp_alpha=0, max_depth=4,
                                       min_samples_leaf=2, min_samples_split=9,
                                       n_estimators=343))])

In [13]:
grid.best_estimator_.fit(X_train, y_train)
r2_score(y_test, grid.best_estimator_.predict(X_test))
#0.5901821334115114

D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [0, 2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


0.5963850181058674

In [14]:
test = pd.read_csv('test.csv')
test

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,8410,aj,h,as,f,d,aa,j,e,0,...,0,0,0,0,0,0,0,0,0,0
4205,8411,t,aa,ai,d,d,aa,j,y,0,...,0,1,0,0,0,0,0,0,0,0
4206,8413,y,v,as,f,d,aa,d,w,0,...,0,0,0,0,0,0,0,0,0,0
4207,8414,ak,v,as,a,d,aa,c,q,0,...,0,0,1,0,0,0,0,0,0,0


In [15]:
X = test.drop(['ID'], axis=1)

In [16]:
results = pd.DataFrame({'ID' : test['ID'], 'y' : grid.best_estimator_.predict(X)})
results

D:\anaconda\envs\merc\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [0, 2, 5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


,ID,y
0,1,77.421789
1,2,94.075113
2,3,77.273862
3,4,77.382459
4,5,112.352965
...,...,...
4204,8410,102.989306
4205,8411,94.250055
4206,8413,94.068007
4207,8414,112.243607


In [18]:
results.to_csv('submission.csv', index=False)